## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Kawalu,ID,-7.3817,108.2082,69.98,91,89,1.05
1,1,Rikitea,PF,-23.1203,-134.9692,72.75,80,100,25.03
2,2,Hasaki,JP,35.7333,140.8333,81.00,85,100,20.27
3,3,Albany,US,42.6001,-73.9662,93.94,52,67,1.01
4,4,Bela,IN,25.9333,81.9833,85.21,70,91,7.78


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
2,2,Hasaki,JP,35.7333,140.8333,81.00,85,100,20.27
4,4,Bela,IN,25.9333,81.9833,85.21,70,91,7.78
5,5,Avarua,CK,-21.2078,-159.7750,77.05,65,40,21.85
6,6,San Francisco,US,37.7749,-122.4194,86.50,70,20,17.27
14,14,Canby,US,45.2629,-122.6926,87.03,32,0,3.00
21,21,Sao Filipe,CV,14.8961,-24.4956,78.62,84,100,7.02
22,22,Sirajganj,BD,24.4500,89.7167,80.96,89,30,15.59
26,26,Yining,CN,29.0361,114.5636,77.54,87,51,2.57
33,33,Butaritari,KI,3.0707,172.7902,80.44,81,86,14.72
34,34,Auki,SB,-8.7676,160.7034,78.15,85,45,4.12


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID       249
City          249
Country       249
Lat           249
Lng           249
Max Temp      249
Humidity      249
Cloudiness    249
Wind Speed    249
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Hasaki,JP,81.00,35.7333,140.8333,
4,Bela,IN,85.21,25.9333,81.9833,
5,Avarua,CK,77.05,-21.2078,-159.7750,
6,San Francisco,US,86.50,37.7749,-122.4194,
14,Canby,US,87.03,45.2629,-122.6926,
21,Sao Filipe,CV,78.62,14.8961,-24.4956,
22,Sirajganj,BD,80.96,24.4500,89.7167,
26,Yining,CN,77.54,29.0361,114.5636,
33,Butaritari,KI,80.44,3.0707,172.7902,
34,Auki,SB,78.15,-8.7676,160.7034,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError, NameError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [14]:
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Hasaki,JP,81.00,35.7333,140.8333,Inubosaki Kanko Hotel
4,Bela,IN,85.21,25.9333,81.9833,Raj Bhawan
5,Avarua,CK,77.05,-21.2078,-159.7750,Paradise Inn
6,San Francisco,US,86.50,37.7749,-122.4194,Edwardian Hotel
14,Canby,US,87.03,45.2629,-122.6926,Canby Country Inn


In [15]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City          249
Country       249
Max Temp      249
Lat           249
Lng           249
Hotel Name    249
dtype: int64

In [22]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_Database.csv" 
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [27]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [28]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))